In [ ]:
!pip install tqdm Pillow


In [ ]:
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 5 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (11.6 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 119604 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-en

In [ ]:
!tesseract --version
import pytesseract

# Tesseract executable path for Google Colab
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'


tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX512BW
 Found AVX512F
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


In [ ]:
import pytesseract
from PIL import Image

def extract_text(image_path):
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return ""


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Assuming extract_text is already defined above

def process_images_from_folder(download_folder, output_csv_path):
    # Initialize an empty list to store the results
    results = []

    # Get all the image filenames in the download folder
    image_filenames = [f for f in os.listdir(download_folder) if os.path.isfile(os.path.join(download_folder, f))]

    # Iterate over each image file to process it
    for image_filename in tqdm(image_filenames):
        image_path = os.path.join(download_folder, image_filename)

        # Check if the image exists (although we know it should since we're listing the folder)
        if os.path.exists(image_path):
            try:
                # Extract text from the image using OCR
                extracted_text = extract_text(image_path)
            except Exception as e:
                print(f"Error processing {image_filename}: {str(e)}")
                extracted_text = ""  # If any error occurs, set extracted text to empty
        else:
            print(f"Image {image_filename} not found!")
            extracted_text = ""

        # Append the result as a dictionary
        results.append({
            'image_filename': image_filename,
            'extracted_text': extracted_text
        })

    # Convert the results to a DataFrame and save it as a CSV
    df_results = pd.DataFrame(results)
    df_results.to_csv(output_csv_path, index=False)

    return df_results

# Example usage
download_folder = './images'  # Replace with the path where images are downloaded
output_csv_path = 'ocr_extracted_texts.csv'

# Run the function to process images
ocr_results_df = process_images_from_folder(download_folder, output_csv_path)
print(ocr_results_df.head())


100%|██████████| 98/98 [02:23<00:00,  1.47s/it]

    image_filename                                     extracted_text
0  71+fn9TWQmL.jpg  SED eee Ss\nPP FO SS es SS\niver 1\nseer os\n\...
1  71aKgRRQ2wL.jpg  Lane SESS SST FSS\nwo gE SS SSSI SE TS TS\nSSN...
2  71SuzaRS7gL.jpg  waz\n\n   \n \n\nDendrobium &Mullein\nExtract ...
3  71XK5d3Oh9L.jpg                                                \n
4  814sAvV89SL.jpg  Nutrition Facts\n\nServing size 3 Pieces (42g)...


In [ ]:
import cv2
import os
import pandas as pd
from matplotlib import pyplot as plt

# Directory where the input images are stored
image_folder = './images/'

# Directory where processed images will be saved
processed_dir = './processed_images/'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Function to display an image
def display(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Function for grayscale conversion
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Function for noise removal
def noise_removal(image):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.medianBlur(image, 3)
    return image

# Function for thinning fonts
def thin_font(image):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    image = cv2.erode(image, kernel, iterations=1)
    return image

# Processing each image from the folder
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)

        if image is not None:
            # Grayscale
            gray_image = grayscale(image)

            # Noise removal
            no_noise_image = noise_removal(gray_image)

            # Thin font
            thinned_image = thin_font(no_noise_image)

            # Save processed image
            output_path = os.path.join(processed_dir, f"processed_{filename}")
            cv2.imwrite(output_path, thinned_image)

            print(f"Processed and saved: {output_path}")
        else:
            print(f"Error loading image: {filename}")


Processed and saved: ./processed_images/processed_71+fn9TWQmL.jpg
Processed and saved: ./processed_images/processed_71aKgRRQ2wL.jpg
Processed and saved: ./processed_images/processed_71SuzaRS7gL.jpg
Processed and saved: ./processed_images/processed_71XK5d3Oh9L.jpg
Processed and saved: ./processed_images/processed_814sAvV89SL.jpg
Processed and saved: ./processed_images/processed_51b9JEHOriL.jpg
Processed and saved: ./processed_images/processed_71nsfFCXF0L.jpg
Processed and saved: ./processed_images/processed_71004c9tzfL.jpg
Processed and saved: ./processed_images/processed_71bvOuz9w1L.jpg
Processed and saved: ./processed_images/processed_41EjbFu-+yL.jpg
Processed and saved: ./processed_images/processed_61WFh8RCQYL.jpg
Processed and saved: ./processed_images/processed_81hnk2WXO3L.jpg
Processed and saved: ./processed_images/processed_817vo3DcCNL.jpg
Processed and saved: ./processed_images/processed_71dWDwMhWmS.jpg
Processed and saved: ./processed_images/processed_71gSRbyXmoL.jpg
Processed 

In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
import tempfile
from skimage import exposure

# Directory for saving processed images
processed_dir = './processed_images12/'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Directory where raw images are stored
image_folder = './images/'

# Function to display an image using matplotlib
def display(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Function for setting DPI (mainly for use with PIL)
def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename

# Function for grayscale conversion
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Function for binarization (thresholding)
def thresholding(image):
    if len(image.shape) == 3:  # Convert color image to grayscale if necessary
        image = get_grayscale(image)
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Function for noise removal
def remove_noise(image):
    if len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
    else:  # Grayscale image
        return cv2.fastNlMeansDenoising(image, None, 10, 7, 21)

# Function for deskewing an image
def deskew(image):
    co_ords = np.column_stack(np.where(image > 0))
    if co_ords.shape[0] == 0:
        print("No contours found to deskew. Returning original image.")
        return image
    angle = cv2.minAreaRect(co_ords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Function for applying CLAHE (Contrast Limited Adaptive Histogram Equalization)
def apply_clahe(image):
    if len(image.shape) == 3:  # Convert color image to grayscale if necessary
        image = get_grayscale(image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# Function for removing borders
def remove_borders(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        print("No contours found. Returning original image.")
        return image
    cnt = max(contours, key=cv2.contourArea)
    if cnt.shape[0] >= 3:  # We need at least 3 points to form a hull
        hull = cv2.convexHull(cnt)
    else:
        print("Contour has insufficient points to form a convex hull. Skipping border removal.")
        return image
    x, y, w, h = cv2.boundingRect(cnt)
    return image[y:y+h, x:x+w]

# Function for image normalization
def normalize_image(image):
    norm_img = np.zeros((image.shape[0], image.shape[1]))
    return cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX)

# Function for perspective transformation (correction of 3D perspective distortions)
def correct_perspective(image):
    # This function is a placeholder; actual implementation will depend on the specific distortions
    # For now, we are not applying any perspective correction
    return image

# Function for image sharpening
def sharpen_image(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# Function for blurring images
def blur_image(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

# Main loop to process each image from the images folder
for image_name in os.listdir(image_folder):
    image_file = os.path.join(image_folder, image_name)

    try:
        # Load the image
        img = cv2.imread(image_file)
        if img is None:
            print(f"Error loading image: {image_file}")
            continue

        # Resize image for DPI (if needed)
        img_dpi_set = set_image_dpi(image_file)

        # Noise removal
        no_noise_image = remove_noise(img)

        # Apply CLAHE
        clahe_image = apply_clahe(no_noise_image)

        # Binarization (thresholding)
        bin_image = thresholding(clahe_image)

        # Deskew the image
        deskewed_image = deskew(bin_image)

        # Normalize the image
        norm_image = normalize_image(deskewed_image)

        # Correct perspective (if needed)
        perspective_corrected_image = correct_perspective(norm_image)

        # Sharpen the image
        sharpened_image = sharpen_image(perspective_corrected_image)

        # Remove borders
        no_border_image = remove_borders(sharpened_image)

        # Save the final processed image
        output_path = os.path.join(processed_dir, f"processed_{image_name}")
        cv2.imwrite(output_path, no_border_image)

        print(f"Processed and saved: {output_path}")

    except Exception as e:
        print(f"Error processing image {image_file}: {e}")


<ipython-input-19-a3d4a773cec3>:28: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im_resized = im.resize(size, Image.ANTIALIAS)


Processed and saved: ./processed_images12/processed_71+fn9TWQmL.jpg
Processed and saved: ./processed_images12/processed_71aKgRRQ2wL.jpg
Processed and saved: ./processed_images12/processed_71SuzaRS7gL.jpg
Processed and saved: ./processed_images12/processed_71XK5d3Oh9L.jpg
Processed and saved: ./processed_images12/processed_814sAvV89SL.jpg
Processed and saved: ./processed_images12/processed_51b9JEHOriL.jpg
Processed and saved: ./processed_images12/processed_71nsfFCXF0L.jpg
Processed and saved: ./processed_images12/processed_71004c9tzfL.jpg
Processed and saved: ./processed_images12/processed_71bvOuz9w1L.jpg
Processed and saved: ./processed_images12/processed_41EjbFu-+yL.jpg
Processed and saved: ./processed_images12/processed_61WFh8RCQYL.jpg
Processed and saved: ./processed_images12/processed_81hnk2WXO3L.jpg
Processed and saved: ./processed_images12/processed_817vo3DcCNL.jpg
Processed and saved: ./processed_images12/processed_71dWDwMhWmS.jpg
Processed and saved: ./processed_images12/proces

In [ ]:
import cv2
import os
import numpy as np
from google.colab import drive
from matplotlib import pyplot as plt
from PIL import Image
import tempfile

# Mount Google Drive to access the images
# drive.mount('/content/drive')

# Directory for saving processed images
processed_dir = './processed_images/'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Directory where raw images are stored
image_folder = './images/'

# Function to display an image using matplotlib
def display(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Function for grayscale conversion
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Function for binarization (thresholding) - ensures image is grayscale
def thresholding(image):
    if len(image.shape) == 3:  # Convert color image to grayscale if necessary
        image = get_grayscale(image)
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Function for noise removal (handles color and grayscale images)
def remove_noise(image):
    if len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
    else:  # Grayscale image
        return cv2.fastNlMeansDenoising(image, None, 10, 7, 21)

# Function for deskewing an image
def deskew(image):
    co_ords = np.column_stack(np.where(image > 0))

    if co_ords.shape[0] == 0:
        print("No contours found to deskew. Returning original image.")
        return image

    angle = cv2.minAreaRect(co_ords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Function to remove borders
def remove_borders(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours are found
    if len(contours) == 0:
        print("No contours found. Returning original image.")
        return image

    # Find the largest contour
    cnt = max(contours, key=cv2.contourArea)

    # Ensure that the contour is in the correct format
    if cnt.shape[0] >= 3:  # We need at least 3 points to form a hull
        hull = cv2.convexHull(cnt)  # Only apply if there are enough points
    else:
        print("Contour has insufficient points to form a convex hull. Skipping border removal.")
        return image

    # Create bounding rectangle around the largest contour
    x, y, w, h = cv2.boundingRect(cnt)
    return image[y:y+h, x:x+w]

# Function to normalize the image
def normalize_image(image):
    norm_img = np.zeros((image.shape[0], image.shape[1]))
    return cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX)

# Function to set DPI (mainly for use with PIL)
def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename

# Main loop to process each image from the images folder
for image_name in os.listdir(image_folder):
    image_file = os.path.join(image_folder, image_name)

    try:
        # Load the image
        img = cv2.imread(image_file)

        if img is None:
            print(f"Error loading image: {image_file}")
            continue

        # Noise removal (using the appropriate method for grayscale or color)
        no_noise_image = remove_noise(img)

        # Grayscale conversion (if not already grayscale)
        gray_image = get_grayscale(no_noise_image)

        # Binarization (thresholding)
        bin_image = thresholding(gray_image)

        # Deskew the image
        deskewed_image = deskew(bin_image)

        # Normalize the image
        norm_image = normalize_image(deskewed_image)

        # Remove borders
        no_border_image = remove_borders(norm_image)

        # Save the final processed image
        output_path = os.path.join(processed_dir, f"processed_{image_name}")
        cv2.imwrite(output_path, no_border_image)

        print(f"Processed and saved: {output_path}")

    except Exception as e:
        print(f"Error processing image {image_file}: {e}")


Processed and saved: ./processed_images/processed_81RNsNEM1EL.jpg
Processed and saved: ./processed_images/processed_71+fn9TWQmL.jpg
Processed and saved: ./processed_images/processed_91Cma3RzseL.jpg
Processed and saved: ./processed_images/processed_81fOxWWWKYL.jpg
Processed and saved: ./processed_images/processed_715vVcWJxGL.jpg
Processed and saved: ./processed_images/processed_71aKgRRQ2wL.jpg
Processed and saved: ./processed_images/processed_71DiLRHeZdL.jpg
Processed and saved: ./processed_images/processed_61BZ4zrjZXL.jpg
Processed and saved: ./processed_images/processed_61oMj2iXOuL.jpg
Processed and saved: ./processed_images/processed_71IUuTJ8QwL.jpg
Processed and saved: ./processed_images/processed_61I9XdN6OFL.jpg
Processed and saved: ./processed_images/processed_51WsuKKAVrL.jpg
Processed and saved: ./processed_images/processed_61wzlucTREL.jpg
Processed and saved: ./processed_images/processed_81S2+GnYpTL.jpg
Processed and saved: ./processed_images/processed_612mrlqiI4L.jpg
Processed 

In [ ]:
import os
import pandas as pd
from PIL import Image
import pytesseract
import re



# Function to preprocess and clean text
def preprocess_text(text):
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    return text

# Function to extract text from an image
def extract_text_from_image(image_path):
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return preprocess_text(text)
    except Exception as e:
        print(f"Error extracting text from {image_path}: {e}")
        return ""

# Directory containing preprocessed images
image_dir = './processed_images_combined'

# Read train.csv
train_df = pd.read_csv('./train.csv')

# Filter train.csv for images that are in the processed images folder
image_filenames = [filename for filename in os.listdir(image_dir) if filename.startswith('processed_')]
filtered_train_df = train_df[train_df['image_link'].apply(lambda x: os.path.basename(x) in image_filenames)]

# Check if filtered_train_df is correctly populated
print(f"Filtered train.csv contains {len(filtered_train_df)} entries")

# Prepare to save results
results = []

# Process each image
for filename in image_filenames:
    image_path = os.path.join(image_dir, filename)
    extracted_text = extract_text_from_image(image_path)

    # Check the extracted text
    print(f"Extracted text from {filename}: {extracted_text}")

    # Extract the entity value
    matched = filtered_train_df[filtered_train_df['image_link'].apply(lambda x: os.path.basename(x) == filename)]
    if not matched.empty:
        for _, row in matched.iterrows():
            entity_value = row['entity_value']
            results.append({
                'image_filename': filename,
                'extracted_text': extracted_text,
                'entity_value': entity_value
            })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_df.to_csv('matched_entities.csv', index=False)

print("Matching complete. Results saved to 'matched_entities.csv'.")


Filtered train.csv contains 0 entries
Extracted text from processed_71ZtDgGX+iL.jpg: a fe htt he co information ————— §& ——_ & chain length: 48+5 cm &* material: titanium steel & size: 20x13 mm &* weight: 8.1 g (about) tips: the size ol the product is measured manually and may be subject lo marginal error 13mm ht mi hh li on hi ot omlhlhcuhc6lt ae
Extracted text from processed_81xsq6vf2qL.jpg: directions: for adults, take two (2) vegan capsules daily, preferably with a meal. do not exceed stated dose. nutrition information high strength typically perdaily dose psyllium husk powder 1400mg ingredients: psyllium husk powder, capsule shell (hy- droxypropylmethylcellulose), anti-caking agents (mag- nesium salts of fatty acids, silicon dioxide). may contain sesame seeds & mustard. for allergens, see the ingredients in bold. notice: take this product with 220ml of fluids. taking this product without adequate fluid may cause the pos- sibility of choking. do not use this product if you have ' 4

In [ ]:
!pip install tensorflow pandas numpy matplotlib
